In [57]:
import numpy as np
import pandas as pd
import pathlib
import os

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [67]:
img_paths = ['data/img2/{}'.format(img) for img in os.listdir('./data/img2/')]

In [71]:
dataset = pd.read_csv('./data/csv/Iris.csv').drop(["Id", "Species"], axis=1)

dataset

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [189]:
data = pd.DataFrame({
    "Filepath": [x for x in img_paths],
    "SepalLenght": dataset["SepalLengthCm"],
    "SepalWidth": dataset["SepalWidthCm"],
    "PetalLength": dataset["PetalLengthCm"],
    "PetalWidth": dataset["PetalWidthCm"]
})

In [178]:
image_df = data.sample(150, random_state=1).reset_index(drop=True)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [179]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [180]:
train_df

,Filepath,SepalLenght,SepalWidth,PetalLength,PetalWidth
118,data/img2/iris-virginica-7.png,6.5,3.0,5.2,2.0
18,data/img2/iris-setosa-38.png,5.4,3.4,1.5,0.4
4,data/img2/iris-virginica-38.png,7.9,3.8,6.4,2.0
45,data/img2/iris-virginica-26.png,7.7,2.6,6.9,2.3
59,data/img2/iris-versicolor-50.png,5.7,3.0,4.2,1.2
...,...,...,...,...,...
133,data/img2/iris-setosa-16.png,5.0,3.4,1.5,0.2
137,data/img2/iris-versicolor-1.png,7.0,3.2,4.7,1.4
72,data/img2/iris-virginica-44.png,6.0,3.0,4.8,1.8
140,data/img2/iris-setosa-32.png,5.0,3.0,1.6,0.2


In [182]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=['SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth'],
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=['SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth'],
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=['SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth'],
    target_size=(256, 256),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 84 validated image filenames.
Found 21 validated image filenames.
Found 45 validated image filenames.


In [183]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(4, activation='relu')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=15,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/15
3/3 [==============================] - 2s 402ms/step - loss: 14.9114 - val_loss: 15.2873
Epoch 2/15
3/3 [==============================] - 1s 329ms/step - loss: 13.9809 - val_loss: 14.1264
Epoch 3/15
3/3 [==============================] - 1s 328ms/step - loss: 12.6684 - val_loss: 12.4542
Epoch 4/15
3/3 [==============================] - 1s 387ms/step - loss: 10.5875 - val_loss: 10.2010
Epoch 5/15
3/3 [==============================] - 1s 329ms/step - loss: 8.2623 - val_loss: 7.3976
Epoch 6/15
3/3 [==============================] - 1s 330ms/step - loss: 5.5545 - val_loss: 4.7475
Epoch 7/15
3/3 [==============================] - 1s 336ms/step - loss: 3.1715 - val_loss: 3.6840
Epoch 8/15
3/3 [==============================] - 1s 386ms/step - loss: 3.0369 - val_loss: 4.1908
Epoch 9/15
3/3 [==============================] - 1s 327ms/step - loss: 3.2708 - val_loss: 3.7106
Epoch 10/15
3/3 [==============================] - 1s 338ms/step - loss: 2.5109 - val_loss: 3.0512
Epoch 11/15

In [170]:
test_images

In [184]:
model.predict(test_images)

2/2 [==============================] - 0s 31ms/step


array([[5.6682987 , 3.038423  , 2.902896  , 1.1685958 ],
       [3.9244907 , 2.1137552 , 2.0205288 , 0.8129194 ],
       [5.8373523 , 3.1286864 , 2.9867575 , 1.2022854 ],
       [6.1702557 , 3.3079958 , 3.1653392 , 1.283871  ],
       [6.0338154 , 3.232147  , 3.0881224 , 1.2423741 ],
       [4.269561  , 2.2971897 , 2.1940234 , 0.8817031 ],
       [7.4181514 , 3.9684095 , 3.7939343 , 1.5264286 ],
       [5.651455  , 3.0293531 , 2.888953  , 1.1728017 ],
       [5.5073996 , 2.9550686 , 2.8276665 , 1.1370959 ],
       [6.449293  , 3.451454  , 3.2997494 , 1.3174871 ],
       [5.7990847 , 3.1103048 , 2.980008  , 1.1937323 ],
       [5.6534395 , 3.031692  , 2.8996503 , 1.1536772 ],
       [5.7362614 , 3.0761456 , 2.9441469 , 1.1856239 ],
       [4.2884645 , 2.3070035 , 2.2047591 , 0.8911199 ],
       [5.2212334 , 2.8024116 , 2.6724603 , 1.0750227 ],
       [7.864077  , 4.2058425 , 4.031617  , 1.5961809 ],
       [5.6589174 , 3.033955  , 2.9020147 , 1.1595869 ],
       [4.7282867 , 2.54143   ,

In [194]:
img = tf.keras.preprocessing.image.load_img('data/img2/iris-setosa-43.png', target_size=(256,256))
img = tf.keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)

model.predict(img)

1/1 [==============================] - 0s 21ms/step


array([[1525.1606 ,  810.01117,  777.0977 ,  311.85217]], dtype=float32)

In [191]:
data.mean()

C:\Users\rodri\AppData\Local\Temp\ipykernel_10444\531903386.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.mean()


SepalLenght    5.843333
SepalWidth     3.054000
PetalLength    3.758667
PetalWidth     1.198667
dtype: float64

In [192]:
data.std()

C:\Users\rodri\AppData\Local\Temp\ipykernel_10444\2723740006.py:1: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.std()


SepalLenght    0.828066
SepalWidth     0.433594
PetalLength    1.764420
PetalWidth     0.763161
dtype: float64